In [31]:
import nltk
import numpy as np
import random
import string
import os.path
from os import path

In [32]:
f=open('/Users/amipatel/Desktop/kb.txt','r',errors = 'ignore')
raw=f.read()
raw=raw.lower()
raw=' '.join(raw.split())

sent_tokens = nltk.sent_tokenize(raw)
word_tokens = nltk.word_tokenize(raw)

In [33]:
inputs = ("hello", "hi", "greetings", "hey")

def greeting(sentence):
 
    for word in sentence.split():
        if word.lower() in inputs:
            return 'Hey, let\'s start with your name. Please introduce yourself'
    return None

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [35]:
from textblob import TextBlob
import re

In [36]:
simpleChat = [['PRP','VBP', 'DT', 'JJ', 'NN'], 
              ['PRP','VBP','DT', 'NN'], 
              ['PRP','VBP', 'JJ'], 
              ['PRP','VBP', 'JJ', 'NN']] 

def find_prp(s):
    response = None
    
    for w,POS in s.pos_tags:
        if w == 'you':
            response = 'I'
        if w == 'i':
            response = 'You'
            
    return response

def find_nn(s):
    nn = None
    
    for w,POS in s.pos_tags:
        if 'NN' in POS:
            nn = w
            
    return nn
    
def find_vp(s):
    vb = None
    
    for w,POS in s.pos_tags:
        if 'VB' in POS:
            if w == 'am':
                vb = 'are'
            if w == 'are':
                vb = 'am'
            else:
                vb = w
            
    return vb

def find_adj(s):
    adj = None
    
    for w,POS in s.pos_tags:
        if 'JJ' in POS:
            adj = w
            
    return adj

def find_dt(s):
    dt = None
    
    for w,POS in s.pos_tags:
        if 'DT' in POS:
            dt = w
            
    return dt

def find_parts_of_speech(parsed):
    for s in parsed.sentences:
        prp_response = find_prp(s)
        nn_response = find_nn(s)
        vb_response = find_vp(s)
        adj_response = find_adj(s)
        dt_response = find_dt(s)
    return prp_response, nn_response, vb_response, adj_response, dt_response
    
def response2(user):
    sent_tokens.append(user)
    userl = user.lower()
    punctuations = '''!()-[]{};:"\,<>/@#$%^&*_~'''
    text = userl
    for x in userl: 
        if x in punctuations: 
            text = userl.replace(x, " ") 
    content = re.sub(r'[^a-z0-9 .?]+', ' ',text)
    parsed = TextBlob(content)
    prp, nn, vb, adj, dt = find_parts_of_speech(parsed)
    pos = []
    for w,POS in parsed.pos_tags:
        pos.append(POS)
    res = ''
    if '?' in user:
        return response(user)
    elif pos in simpleChat:
        res = 'Sure, '
        if parsed.noun_phrases is not []:
            if nn is not None:
                if dt is None and adj is not None:
                    res += prp + ' ' + vb + ' ' + adj + ' ' + nn + '!'
                elif dt is None and adj is None:
                    res += prp + ' ' + vb + ' ' + nn + '!'
                elif dt is not None and adj is None:
                    res += prp + ' ' + vb + ' ' + dt + ' ' + nn + '!'
                else:
                    res += prp + ' ' + vb + ' ' + dt + ' ' + adj + ' ' + nn + '!'
            else:
                if dt is None and adj is not None:
                    res += prp + ' ' + vb + ' ' + adj + '!'
                elif dt is None and adj is None:
                    res += prp + ' ' + vb + '!'
                elif dt is not None and adj is None:
                    res += prp + ' ' + vb + ' ' + dt + '!'
                else:
                    res += prp + ' ' + vb + ' ' + dt + ' ' + adj + '!'
        elif adj is not None:
            res += prp + ' ' + vb + ' ' + adj + '!'
        else:
            if dt is not None and nn is not None:
                res += prp + ' ' + vb + ' ' + dt + ' ' + nn + '!'
        sent_tokens.remove(user)
        return res
    else:
        return response(user)
    

In [39]:
def response(user_response):
    bot_response=''
    user_response = user_response.lower()
    punctuations = '''!()-[]{};:"\,<>/@#$%^&*_~'''
    for x in user_response: 
        if x in punctuations: 
            user_response = user_response.replace(x, " ") 
    content = re.sub(r'[^a-z0-9 .?]+', ' ',user_response)
    question = False
    use = ''    
    if '?' in content:
        if(content=='how are you?' or content=='what is your name?'):  
            if content=='how are you?':
                bot_response = 'I am fine!'
            else:
                bot_response = 'My name is DIETO. I will answer your queries about diets.'
            sent_tokens.remove(user_response)
            return bot_response
    
    TfidfVec = TfidfVectorizer()
    tfidf = TfidfVec.fit_transform(sent_tokens)
    similar_vector_vals = cosine_similarity(tfidf[-1], tfidf)
    
    # the second last sentence will have the highest cosine similarity
    if question is True:
        sentence_number=similar_vector_vals.argsort()[0][-3]  
    else:
        sentence_number=similar_vector_vals.argsort()[0][-2]
    
    # Flatten to check if the cosine similarity is 0 or not
    flat = similar_vector_vals.flatten()
    flat.sort()
    actual_tfidf = flat[-2]
    if(actual_tfidf==0):
        bot_response=bot_response+"I am sorry! I don't understand you"
        sent_tokens.remove(user_response)
        return bot_response
    else:
        bot_response = bot_response+sent_tokens[sentence_number]
        if question is True:
            sent_tokens.remove(use)
        sent_tokens.remove(user_response)
        return bot_response

In [40]:
Bye = False
print("DIETO: My name is DIETO. I will answer your queries about diets. If you want to exit, type Bye!")
file = 'users.txt'
user_data = []
while(Bye is not True):
    print("YOU: ", end = '')
    user_response = input()
    user_response = user_response.lower()
    user_data.append(user_response)
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            print("DIETO: You are welcome.")
        else:
            if(greeting(user_response)!=None):
                respond = greeting(user_response)
                print("DIETO: " + respond)
                user_data.append(respond)
                print("YOU: ", end = '')
                user_response = input()
                user_data.append(user_response)
                if(user_response!='bye'):
                    p = TextBlob(user_response)
                    names = []
                    if path.exists(file):
                        f = open(file, 'r') 
                        for line in f:
                            if ':' in line:
                                data = line.split(':')
                                name = data[0]
                                names.append(name)
                        f.close()
                    new_name = p.noun_phrases
                    if new_name[0] not in names:
                        new = True
                        respond = "Nice to meet you, " + new_name[0] + "! Do you want to know more about healthy diet plans?"
                        print("DIETO: " + respond)
                        user_data.append(respond)
                        print("YOU: ", end = '')
                        user_response = input()
                        user_response.lower()
                        if 'yes' in user_response:
                            user_data.append('LIKE-' + user_response)
                            respond = "Perfect! Let's get started. You can ask me any questions you might have!"
                            print("DIETO: " + respond)
                            user_data.append(respond)
                            print("YOU: ", end = '')
                            user_response = input()
                            user_data.append(user_response)
                            user_response.lower()
                            if 'okay' in user_response or 'yeah' in user_response:
                                respond = "Glad to know you are interested in talking about healthy foods! Shoot me your questions.."
                                print("DIETO: " + respond)
                                user_data.append(respond)
                            else:
                                print("DIETO: ",end="")
                                respond = response2(user_response)
                                print(respond)
                                user_data.append(respond)
                        elif 'no' in user_response:
                            user_data.append('DISLIKE-' + user_response)
                            respond = "I will try my best to make it more intersting for you! Shoot me your questions.."
                            print("DIETO: " + respond)
                            user_data.append(respond)
                    else:
                        new = False
                        dislike = False
                        flag = False
                        f = open(file, 'r') 
                        for line in f:
                            if flag is True:
                                if ':' not in line:
                                    if 'DISLIKE-' in line:
                                        dislike = True
                                        break
                                else:
                                    flag = False
                            else:
                                if ':' in line:
                                    p = line.split(':')
                                    name = p[0]
                                    if name == new_name[0]:
                                        flag = True
                        f.close()
                        respond = "Welcome back, " + new_name[0] + "!"
                        if dislike is True:
                            respond += " I will try my best to make it more intersting for you! Shoot me your questions.."
                        else:
                            respond += " Glad to know you are interested in talking about healthy foods! Shoot me your questions.."
                        print("DIETO: " + respond)
                        if dislike is True:
                            user_data.append('DISLIKE-' + respond)
                        else:
                            user_data.append('LIKE-' + respond)
                else:
                    Bye = True
                    print("DIETO: Bye! take care..")  
            else:
                if 'sure' in user_response or 'yeah' in user_response or 'okay'in user_response:
                    respond = "Please feel free to ask me any further questions you might have! If you want to exit, type Bye!"
                    print("DIETO: " + respond)
                    user_data.append(respond) 
                    continue
                    
                print("DIETO: ",end="")
                respond = response2(user_response)
                print(respond)
                user_data.append(respond)
                
    else:
        Bye = True
        print("DIETO: Bye! take care..")
        if new is True:
            with open(file, 'a+') as f:
                line = new_name[0] + ':' + '\n'
                for data in user_data:
                    line += data + '\n'
                f.write(line)
        else:
            f = open(file, 'r') 
            lines = f.read().split('\n')
            f.close()
            f = open(file, 'w') 
            flag = False
            for line in lines:
                if flag is True:
                    if ':' not in line:
                        continue
                    else:
                        f.write(line + '\n')
                        flag = False
                else: 
                    if ':' in line:
                        p = line.split(':')
                        name = p[0]
                        if name == new_name[0]:
                            d = new_name[0] + ':' + '\n'
                            for data in user_data:
                                d += data + '\n'
                            f.write(d)
                            flag = True
                        else:
                            flag = False
                            d = name + ':\n'
                            f.write(d)
                    else:
                        f.write(line + '\n')
            f.close()

DIETO: My name is DIETO. I will answer your queries about diets. If you want to exit, type Bye!
YOU: Hi
DIETO: Hey, let's start with your name. Please introduce yourself
YOU: my name is Ami
DIETO: Welcome back, ami! Glad to know you are interested in talking about healthy foods! Shoot me your questions..
YOU: what is atkins diet?
DIETO: atkins diet: phase 1how much weight you can lose on atkins induction phasewhat you can expect for the first two weeks of inductionpros and cons of atkin diethow to start the atkins diet inductionfoods to eat on atkins diet inductionfoods to avoid on atkins diet inductiona sample atkins menu for one weekwhere to get the best atkins diet books what is atkins diet thе atkinѕ diеt iѕ a lоw саrbоhуdrаtе diet recommended fоr wеight lоѕѕ.
YOU: what is diet?
DIETO: the reason for weigth increase is not what or how much you eat that causes weight gain but rather when and in what combinations the food is eaten.
YOU: what is ketosis?
DIETO: the more serious questi